In [ ]:
!pip install -q -U torch torchvision torchaudio transformers datasets accelerate scikit-learn pandas

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Cấu hình thiết bị
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
MODEL_NAME = "vinai/phobert-large"
OUTPUT_DIR = "./phobert-large-hallu-finetuned"

# Đường dẫn gốc tới folder chứa dữ liệu đã xử lý (Dựa theo ảnh bạn gửi)
# Giả sử cấu trúc là: /kaggle/input/processed-hal/processed_data_3labels/{Tên_Bộ}/train.csv
DATA_ROOT = "/kaggle/input/processed-hal/processed_data_3labels/"

# Định nghĩa Mapping từ Chữ sang Số
label2id = {
    "Entailment": 0,
    "Intrinsic-Hal": 1,
    "Extrinsic-Hal": 2
}
id2label = {0: "Entailment", 1: "Intrinsic-Hal", 2: "Extrinsic-Hal"}

def load_and_map_data(dataset_name, split):
    # Tạo đường dẫn file
    file_path = os.path.join(DATA_ROOT, dataset_name, f"{split}.csv")
    
    if not os.path.exists(file_path):
        print(f"⚠️ Không tìm thấy file: {file_path}")
        return None
    
    print(f"🔄 Đang đọc {dataset_name} - {split}...")
    df = pd.read_csv(file_path)
    
    # Lọc bỏ dòng lỗi (nếu có)
    df = df.dropna(subset=['sentence1', 'sentence2', 'label'])
    
    # --- MAP NHÃN TRỰC TIẾP ---
    # Vì file csv đã có nhãn đúng tên, ta chỉ cần map sang số
    df['labels'] = df['label'].map(label2id)
    
    # Kiểm tra xem có nhãn nào lạ không (bị NaN sau khi map)
    if df['labels'].isnull().any():
        print(f"⚠️ Cảnh báo: Có {df['labels'].isnull().sum()} dòng nhãn lạ, sẽ bị bỏ qua.")
        df = df.dropna(subset=['labels'])
        
    df['labels'] = df['labels'].astype(int)
    
    return df[['sentence1', 'sentence2', 'labels']]

# --- LOAD VÀ GỘP 3 BỘ DỮ LIỆU ---
dfs_train = []
dfs_dev = []

# Danh sách tên thư mục con của 3 bộ dữ liệu
datasets_list = ["ViMedNLI", "ViANLI", "ViNLI"] 

for ds in datasets_list:
    t = load_and_map_data(ds, "train")
    d = load_and_map_data(ds, "dev")
    if t is not None: dfs_train.append(t)
    if d is not None: dfs_dev.append(d)

if not dfs_train:
    raise ValueError("❌ LỖI: Không load được dữ liệu nào! Kiểm tra lại đường dẫn DATA_ROOT.")

# Gộp DataFrame
full_train_df = pd.concat(dfs_train, ignore_index=True)
full_dev_df = pd.concat(dfs_dev, ignore_index=True)

# Chuyển sang HuggingFace Dataset
train_dataset = Dataset.from_pandas(full_train_df)
dev_dataset = Dataset.from_pandas(full_dev_df)

print("-" * 40)
print(f"✅ Tổng mẫu Train: {len(train_dataset)}")
print(f"✅ Tổng mẫu Dev: {len(dev_dataset)}")
print(f"✅ Phân bố nhãn Train: {full_train_df['labels'].value_counts().to_dict()}")
print(f"   (0: Entailment, 1: Intrinsic-Hal, 2: Extrinsic-Hal)")
print("-" * 40)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    return tokenizer(
        examples["sentence1"], 
        examples["sentence2"], 
        truncation=True, 
        padding=False, 
        max_length=256 
    )

print("⏳ Đang Tokenize...")
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_dev = dev_dataset.map(preprocess_function, batched=True)

# Xóa cột cũ
cols_to_remove = ['sentence1', 'sentence2']
if '__index_level_0__' in tokenized_train.column_names: cols_to_remove.append('__index_level_0__')
tokenized_train = tokenized_train.remove_columns(cols_to_remove)
tokenized_dev = tokenized_dev.remove_columns(cols_to_remove)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3, # 3 Lớp
    id2label=id2label,
    label2id=label2id
).to(device)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    return {"accuracy": acc, "f1_macro": f1}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=7,              
    learning_rate=2e-5,              
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,   
    
    # --- CẤU HÌNH LƯU MODEL TIẾT KIỆM ---
    eval_strategy="epoch",           # Vẫn đánh giá sau mỗi epoch
    save_strategy="epoch",           # Lưu sau mỗi epoch
    save_total_limit=1,              # <--- QUAN TRỌNG: Chỉ giữ lại 1 checkpoint tốt nhất, xóa các cái cũ
    load_best_model_at_end=True,     # Load model tốt nhất khi xong
    # ------------------------------------
    
    metric_for_best_model="f1_macro",
    weight_decay=0.01,
    report_to="none",
    fp16=True,                       
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Xóa bớt cache cũ nếu có để giải phóng dung lượng ngay lập tức
import shutil
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
    os.makedirs(OUTPUT_DIR)

print("🚀 Bắt đầu Training PhoBERT-Large (3-Class)...")
trainer.train()

In [ ]:
# Lưu model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Model đã lưu tại: {OUTPUT_DIR}")

# Đánh giá chi tiết
print("\n--- Classification Report (Dev Set) ---")
preds_output = trainer.predict(tokenized_dev)
y_preds = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

target_names = ["Entailment", "Intrinsic-Hal", "Extrinsic-Hal"]
print(classification_report(y_true, y_preds, target_names=target_names, digits=4))

# Lưu kết quả ra CSV
df_res = pd.DataFrame({"True": y_true, "Pred": y_preds})
df_res.to_csv("phobert_3class_results.csv", index=False)